In [ ]:
import os
import json
files_config = os.path.abspath('../AccessControl/CVE/all_files.json')
base_path = os.path.abspath('../AccessControl/CVE')

with open(files_config) as f:
    files = json.load(f)
print (files)

In [ ]:
experiment_script_path = os.path.abspath('../RunScripts/run_experiment.sh')
import subprocess
import multiprocessing
from multiprocessing import Pool

# default_timeout = 600
# tool=$1
# file=$2
# timeout=$3
# num_processes=$4
# output_folder=$5
# solc_version=$6
# to_date=$7 (optional, for spcon)
def compile_contract(files_json):
    ... # compile contract in json format

def search_for_bin(abs_file_path, json_data):
    path_parts = abs_file_path.split("/")
    path_parts[-1] = json_data.get("contract")[0] +  ".bin-runtime"
    return "/".join(path_parts)

def get_commands_to_run_tool(json_data, script_path, base_output_path, timeout=60, tools = "all"):
    commands = []
    abs_file_path = os.path.join(base_path, json_data.get("file"))
    binary_file_path = search_for_bin(abs_file_path, json_data)
    # check if bin file exists
    if not os.path.exists(binary_file_path):
        raise Exception("Binary file not found, please compile contract first")
    if "mythril" in tools or tools == "all":
        commands.append(f'{script_path} mythril {abs_file_path} {timeout} 1 {base_output_path}/mythril {json_data.get("version")}')
    if "slither" in tools or tools == "all":
        commands.append(f'{script_path} slither {abs_file_path} {timeout} 1 {base_output_path}/slither {json_data.get("version")}')
    if "spcon" in tools or tools == "all":
        commands.append(f'{script_path} spcon {json_data.get("address")} {timeout} 1 {base_output_path}/ {json_data.get("version")}')
    if "achecker" in tools or tools == "all":
        commands.append(f'{script_path} achecker {binary_file_path} {timeout} 1 {base_output_path}/ {json_data.get("version")}')
    return commands
# Running the bash script with arguments
# subprocess.run(command, shell=True)
def run_command(command):
    try:
        # Execute the command and wait for it to complete
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return command, result.stdout.decode(), result.stderr.decode()
    except subprocess.CalledProcessError as e:
        # Return standard error output if the command fails
        print ("error ", e)
        return command, e.stdout.decode(), e.stderr.decode()

def execute_commands(commands):
    try:
        # Create a pool of worker processes
        with Pool(processes=min(len(commands), multiprocessing.cpu_count())) as pool:
            # Execute the commands in parallel and wait for all to complete
            results = pool.map(run_command, commands)
            print("All commands have been executed.")
            return results
    except Exception as e:
        print(f"An error occurred during command execution: {e}")

def run_experiment(files_json):
    for key_,val_ in files_json.items():
        print (key_,val_)
        file_path = os.path.join(base_path, val_.get("file"))

        print (file_path)
        output_path = os.path.join(base_path, f'tool_output/{val_.get("file")}')
        # print (output_path)
        folder_path = os.path.dirname(file_path)
        found_bin = False
        for file in os.listdir(folder_path):
            if file.endswith('.bin-runtime'):
                found_bin = True
                break
        if not found_bin:
            print ("bin not found")
            raise Exception("Need to compile contract first for some tools")
            # compile_contract(files_json)

        print ("start running experiment")
        commands = get_commands_to_run_tool(val_, experiment_script_path, output_path, timeout=60, tools = "all")
        for command in commands:
            print (command)
        res = execute_commands(commands)
        break
        print ("end running experiment")

run_experiment(files)